In [3]:
import json
import pandas as pd
import requests

In [4]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [5]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [6]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [7]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [8]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [9]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [10]:
access_token = get_access_token(client_key, client_secret)

keeping track with calls
27.02 - we ran keys2 unril 1000 


In [11]:
reposted_videos = []
for username in usernames[1001:2002]:
    reposted_videos.append(get_repost_info(username, access_token))

In [12]:
reposted_videos

[{'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 6268,
    'id': 7473895906744585503,
    'username': 'torimooore'},
   {'username': 'ileana.justine',
    'favorites_count': 62895,
    'id': 7466543229958212906},
   {'favorites_count': 3,
    'id': 7440090648436428062,
    'username': 'cassiesshop'},
   {'username': 'dudeitsmeduh',
    'favorites_count': 33153,
    'id': 7474377350432574763},
   {'favorites_count': 14676,
    'id': 7445235748162243886,
    'username': 'kentuckyfriedfart'},
   {'favorites_count': 166277,
    'id': 7463002562782858538,
    'username': 'haileyshevitz'},
   {'username': 'lurnokeefiee',
    'favorites_count': 39930,
    'id': 7461357804813585707},
   {'username': 'soupgirl228',
    'favorites_count': 1771,
    'id': 7437311074312981790},
   {'username': 'juliacarolann',
    'favorites_count': 559,
    'id': 7457368254923803950},
   {'id': 7449509352299416878,
    'username': 'itsmejadeb',
    'favorites_count': 855},
   {'favori

In [13]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [14]:
combined = {}
for i, username in enumerate(usernames[1001:2002]):
    combined[username] = reposted_videos[i]
    
combined 


{'bethfrombklyn': {'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 6268,
    'id': 7473895906744585503,
    'username': 'torimooore'},
   {'username': 'ileana.justine',
    'favorites_count': 62895,
    'id': 7466543229958212906},
   {'favorites_count': 3,
    'id': 7440090648436428062,
    'username': 'cassiesshop'},
   {'username': 'dudeitsmeduh',
    'favorites_count': 33153,
    'id': 7474377350432574763},
   {'favorites_count': 14676,
    'id': 7445235748162243886,
    'username': 'kentuckyfriedfart'},
   {'favorites_count': 166277,
    'id': 7463002562782858538,
    'username': 'haileyshevitz'},
   {'username': 'lurnokeefiee',
    'favorites_count': 39930,
    'id': 7461357804813585707},
   {'username': 'soupgirl228',
    'favorites_count': 1771,
    'id': 7437311074312981790},
   {'username': 'juliacarolann',
    'favorites_count': 559,
    'id': 7457368254923803950},
   {'id': 7449509352299416878,
    'username': 'itsmejadeb',
    'favorites_count': 

In [15]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

                   username             video_id                creator  \
0             bethfrombklyn  7473895906744585503             torimooore   
1             bethfrombklyn  7466543229958212906         ileana.justine   
2             bethfrombklyn  7440090648436428062            cassiesshop   
3             bethfrombklyn  7474377350432574763           dudeitsmeduh   
4             bethfrombklyn  7445235748162243886      kentuckyfriedfart   
...                     ...                  ...                    ...   
10475  theheartfulhomemaker  7427550330524683563            politipeach   
10476  theheartfulhomemaker  7436521470026616110  that.little.homemaker   
10477  theheartfulhomemaker  7451246814176611630          adambirkmusic   
10478  theheartfulhomemaker  7434953733231938858        emilylynntaylor   
10479  theheartfulhomemaker  7435458526778084639        bridget_lindner   

       favourites  
0            6268  
1           62895  
2               3  
3           33153  

In [16]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)